In [1]:
import numpy as np
import os
import glob
import cv2

import tensorflow as tf
import keras

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization


# Import the backend
from keras import backend as K

import time

Using TensorFlow backend.


In [2]:
train_img = []
train_label = []
for path in glob.glob("../Fazel/fruits-360/Training/*"):
    img_label = path.split("/")[-1]
    for image in glob.glob(os.path.join(path, "*.jpg")):
        img = cv2.imread(image) # the order of colors is BGR when an image file is read by imread()
        img = cv2.resize(img, (64, 64)) # size reduction for purpose of having enough memory
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB because the order of colors is assumed to be RGB
        train_img.append(img)
        train_label.append(img_label)
train_img = np.array(train_img)
train_label = np.array(train_label)
len(np.unique(train_label))

60

In [3]:
test_img = []
test_label =[]
for path in glob.glob("../Fazel/fruits-360/Test/*"):
    img_label = path.split("/")[-1]
    for image in glob.glob(os.path.join(path, "*.jpg")):
        img = cv2.imread(image) # the order of colors is BGR when an image file is read by imread()
        img = cv2.resize(img, (64, 64)) # size reduction for purpose of having enough memory
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB because the order of colors is assumed to be RGB
        test_img.append(img)
        test_label.append(img_label)
test_img = np.array(test_img)
test_label = np.array(test_label)
len(np.unique(test_label))

60

In [4]:
print(train_img.shape)
print(test_img.shape)

(29586, 64, 64, 3)
(10102, 64, 64, 3)


In [5]:
# Label train set
id = {v:k for k,v in enumerate(np.unique(train_label)) }
target = {v:k for k,v in id.items() }
print(target)

{0: 'Training\\Apple Braeburn', 1: 'Training\\Apple Crimson Snow', 2: 'Training\\Apple Golden 1', 3: 'Training\\Apple Granny Smith', 4: 'Training\\Apple Pink Lady', 5: 'Training\\Apple Red 1', 6: 'Training\\Apple Red Yellow 1', 7: 'Training\\Apricot', 8: 'Training\\Avocado', 9: 'Training\\Avocado ripe', 10: 'Training\\Banana', 11: 'Training\\Beetroot', 12: 'Training\\Blueberry', 13: 'Training\\Cantaloupe 1', 14: 'Training\\Cauliflower', 15: 'Training\\Cherry 1', 16: 'Training\\Chestnut', 17: 'Training\\Clementine', 18: 'Training\\Cocos', 19: 'Training\\Dates', 20: 'Training\\Eggplant', 21: 'Training\\Ginger Root', 22: 'Training\\Grape Blue', 23: 'Training\\Grape Pink', 24: 'Training\\Grape White', 25: 'Training\\Grapefruit White', 26: 'Training\\Hazelnut', 27: 'Training\\Huckleberry', 28: 'Training\\Kaki', 29: 'Training\\Kiwi', 30: 'Training\\Kumquats', 31: 'Training\\Lemon', 32: 'Training\\Limes', 33: 'Training\\Mandarine', 34: 'Training\\Mango', 35: 'Training\\Mango Red', 36: 'Traini

In [6]:
# Place train labels in a numpy array
label_id = np.array([id[i] for i in train_label])
print(label_id)
print(train_img.shape, label_id.shape)

[ 0  0  0 ... 59 59 59]
(29586, 64, 64, 3) (29586,)


In [7]:
# Label test set
test_id = {v:k for k,v in enumerate(np.unique(test_label)) }
test_target = {v:k for k,v in test_id.items() }
print(test_target)

{0: 'Test\\Apple Braeburn', 1: 'Test\\Apple Crimson Snow', 2: 'Test\\Apple Golden 1', 3: 'Test\\Apple Granny Smith', 4: 'Test\\Apple Pink Lady', 5: 'Test\\Apple Red 1', 6: 'Test\\Apple Red Yellow 1', 7: 'Test\\Apricot', 8: 'Test\\Avocado', 9: 'Test\\Avocado ripe', 10: 'Test\\Banana', 11: 'Test\\Beetroot', 12: 'Test\\Blueberry', 13: 'Test\\Cantaloupe 1', 14: 'Test\\Cauliflower', 15: 'Test\\Cherry 1', 16: 'Test\\Chestnut', 17: 'Test\\Clementine', 18: 'Test\\Cocos', 19: 'Test\\Dates', 20: 'Test\\Eggplant', 21: 'Test\\Ginger Root', 22: 'Test\\Grape Blue', 23: 'Test\\Grape Pink', 24: 'Test\\Grape White', 25: 'Test\\Grapefruit White', 26: 'Test\\Hazelnut', 27: 'Test\\Huckleberry', 28: 'Test\\Kaki', 29: 'Test\\Kiwi', 30: 'Test\\Kumquats', 31: 'Test\\Lemon', 32: 'Test\\Limes', 33: 'Test\\Mandarine', 34: 'Test\\Mango', 35: 'Test\\Mango Red', 36: 'Test\\Mulberry', 37: 'Test\\Nectarine', 38: 'Test\\Nut Pecan', 39: 'Test\\Onion Red', 40: 'Test\\Onion White', 41: 'Test\\Orange', 42: 'Test\\Papaya',

In [8]:
# Place test labels in a numpy array
test_label_id = np.array([test_id[i] for i in test_label])
print(test_label_id)
print(test_img.shape, test_label_id.shape)

[ 0  0  0 ... 59 59 59]
(10102, 64, 64, 3) (10102,)


In [9]:
X_train = train_img
X_train_2D = X_train.reshape(train_img.shape[0],64*64*3)
print(X_train.shape)
print(X_train_2D.shape)

# Normalize values to 1 from 0 to 255 (256 values of pixels)
X_train = X_train/255

(29586, 64, 64, 3)
(29586, 12288)


In [10]:
X_test = test_img
X_test_2D = test_img.reshape(test_img.shape[0],64*64*3)
print(X_test.shape)
print(X_test_2D.shape)

# Normalize values to 1 from 0 to 255 (256 values of pixels)
X_test = X_test/255

(10102, 64, 64, 3)
(10102, 12288)


In [11]:
Y_train = label_id
Y_test = test_label_id

# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, 60)
Y_test = keras.utils.to_categorical(Y_test, 60)

# Apply CNN Model

In [12]:
# Building the Model
model = Sequential()
# First convolutional layer
model.add(Conv2D(16,(3,3),input_shape=(64,64,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(64,(3,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(60, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)       

In [13]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adamax(),
              metrics=['accuracy'])

In [14]:
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
end_time = time.time()
print('Run time:', end_time-start_time)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 29586 samples, validate on 10102 samples
Epoch 1/10
29586/29586 [==============================] - 874s 30ms/step - loss: 1.3987 - accuracy: 0.6248 - val_loss: 18.2125 - val_accuracy: 0.0164
Epoch 2/10
29586/29586 [==============================] - 990s 33ms/step - loss: 0.3397 - accuracy: 0.8964 - val_loss: 7.6538 - val_accuracy: 0.0384
Epoch 3/10
29586/29586 [==============================] - 1115s 38ms/step - loss: 0.1706 - accuracy: 0.9487 - val_loss: 0.6715 - val_accuracy: 0.8101
Epoch 4/10
29586/29586 [==============================] - 1072s 36ms/step - loss: 0.1128 - accuracy: 0.9672 - val_loss: 0.3560 - val_accuracy: 0.8884
Epoch 5/10
29586/29586 [==============================] - 1019s 34ms/step - loss: 0.0808 - accuracy: 0.9769 - val_loss: 0.3023 - val_accuracy: 0.9122
Epoch 6/10
29586/29586 [==============================] - 1077s 36ms/step - loss: 0.0615 - accuracy: 0.9827 - val_loss: 0.1787 - val_accuracy: 0.9471
Epoch 7/10
29586/29586 [===========================

The Convolutional Neural Networl model had about three with hours run time and an accuracy of almost 95%.

# RNN

In [ ]:
# Embedding dimensions.
row_hidden = 128
col_hidden = 128

row, col, pixel = X_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(60, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(X_test, Y_test))

# Evaluation.
scores = model.evaluate(X_test, Y_test, verbose=0)
end_time = time.time()
print('Run time:', end_time-start_time)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 29586 samples, validate on 10102 samples
Epoch 1/5
 2304/29586 [=>............................] - ETA: 98:39:32 - loss: 4.0895 - accuracy: 0.0234